In [1]:
%cd /work/vajira/DL/singan-polyp-aug

/work/vajira/DL/singan-polyp-aug


In [2]:
import torch
from config import get_arguments
import pathlib
import glob
import argparse
import sys
from PIL import Image
from skimage import io as img

from SinGAN import functions
from SinGAN import manipulate

#from .singan_polyp_aug_temp.SinGAN import functions
#from .singan_polyp_aug_temp.SinGAN import manipulate

In [3]:
Gs_path = "/work/vajira/DL/sinGAN/SinGAN-aug/TrainedModels/cju2u73dj53oz0878486k8k4b/scale_factor=0.750000,alpha=10/Gs.pth"
noise_path = "/work/vajira/DL/sinGAN/SinGAN-aug/TrainedModels/cju2u73dj53oz0878486k8k4b/scale_factor=0.750000,alpha=10/NoiseAmp.pth"
reals_path = "/work/vajira/DL/sinGAN/SinGAN-aug/TrainedModels/cju2u73dj53oz0878486k8k4b/scale_factor=0.750000,alpha=10/reals.pth"
z_path = "/work/vajira/DL/sinGAN/SinGAN-aug/TrainedModels/cju2u73dj53oz0878486k8k4b/scale_factor=0.750000,alpha=10/Zs.pth"


In [4]:
parser = get_arguments()
parser.add_argument('--input_dir', help='input image dir', default='/work/vajira/DATA/hyper_kvasir/data_new/segmented_train_val/data/img_and_mask_together')
parser.add_argument('--input_name', help='input image name', default="cju2u73dj53oz0878486k8k4b.png")
parser.add_argument('--mode', help='random_samples | random_samples_arbitrary_sizes', default='random_samples')
# for random_samples:
parser.add_argument('--gen_start_scale', type=int, help='generation start scale', default=8)
# for random_samples_arbitrary_sizes:
parser.add_argument('--scale_h', type=float, help='horizontal resize factor for random samples', default=1.5)
parser.add_argument('--scale_v', type=float, help='vertical resize factor for random samples', default=1)
opt = parser.parse_args("")
opt = functions.post_config(opt)

Random Seed:  4318


In [5]:
opt.gen_start_scale = 0
opt.nc_z = 4
opt.nc_im = 4
opt.device = torch.device("cpu" if opt.not_cuda else "cuda:0")

In [6]:
#opt = functions.post_config(opt)

In [7]:
def read_image(path):
    x = img.imread(path)

    print("X shape=", x.shape)
    if len(x.shape) == 2: # executee if imag is gray
        x = color.gray2rgb(x)
    print("x after imread===", x.shape)
    x = functions.np2torch(x,opt)
    #print("x after np2torch===", x.shape)
    #print(" x.shape[1]=", x.shape[1])
    if x.shape[1] == 4:
        x = x[:,0:4,:,:]    
    else:
        x = x[:,0:3,:,:]

    return x

In [8]:
real = read_image("/work/vajira/DATA/hyper_kvasir/data_new/segmented_train_val/data/img_and_mask_together/cju2u73dj53oz0878486k8k4b.png")

X shape= (529, 524, 4)
x after imread=== (529, 524, 4)


In [9]:
functions.adjust_scales2image(real, opt)

tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],

         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., 

In [10]:
opt

Namespace(Dsteps=3, Gsteps=3, alpha=10, beta1=0.5, device=device(type='cuda', index=0), gamma=0.1, gen_start_scale=0, input_dir='/work/vajira/DATA/hyper_kvasir/data_new/segmented_train_val/data/img_and_mask_together', input_name='cju2u73dj53oz0878486k8k4b.png', ker_size=3, lambda_grad=0.1, lr_d=0.0005, lr_g=0.0005, manualSeed=4318, max_size=250, min_nfc=32, min_nfc_init=32, min_size=25, mode='random_samples', nc_im=4, nc_z=4, netD='', netG='', nfc=32, nfc_init=32, niter=2000, niter_init=2000, noise_amp=0.1, noise_amp_init=0.1, not_cuda=0, num_layer=5, num_scales=12, out='Output', out_='TrainedModels/cju2u73dj53oz0878486k8k4b/scale_factor=0.750000/', padd_size=0, scale1=0.4725897920604915, scale_factor=0.7749549932130969, scale_factor_init=0.75, scale_h=1.5, scale_v=1, stop_scale=9, stride=1)

In [11]:
Gs = torch.load(Gs_path)
reals = torch.load(reals_path)
Zs = torch.load(z_path)
NoiseAmp = torch.load(noise_path)

In [12]:
len(reals)

10

In [13]:
len(Gs)

10

In [14]:
in_s = functions.generate_in2coarsest(reals, 1, 1, opt)

In [15]:
manipulate.SinGAN_generate(Gs, Zs, reals, NoiseAmp, opt , gen_start_scale=8)

tensor([[[[-0.9166, -0.8533, -0.8701,  ..., -0.9487, -1.0462, -0.9339],
          [-0.8807, -0.8813, -0.8674,  ..., -0.8740, -0.9302, -0.9890],
          [-0.9042, -0.8986, -0.9026,  ..., -0.9078, -0.8853, -0.8908],
          ...,
          [-0.9600, -0.9363, -0.9221,  ..., -0.8668, -0.8497, -0.8173],
          [-1.0106, -0.9762, -0.9651,  ..., -0.9042, -0.9264, -0.8945],
          [-1.0235, -1.0198, -0.9779,  ..., -0.9236, -0.9256, -0.9675]],

         [[-0.9774, -0.9631, -0.9107,  ..., -0.9897, -1.0670, -0.9739],
          [-0.9506, -1.0010, -0.9588,  ..., -0.9609, -1.0322, -1.0523],
          [-0.9386, -0.9854, -0.9928,  ..., -0.9698, -0.9936, -0.9883],
          ...,
          [-1.0153, -1.0065, -0.9793,  ..., -0.9235, -0.9703, -0.9306],
          [-1.0799, -1.0564, -1.0257,  ..., -0.9639, -1.0236, -0.9661],
          [-1.0833, -1.0815, -1.0430,  ..., -0.9806, -0.9971, -1.0000]],

         [[-0.9792, -1.0054, -0.9976,  ..., -0.9969, -1.0803, -0.9862],
          [-0.9450, -1.0136, -

In [16]:
img = Image.open("/work/vajira/DATA/sinGAN_polyps/test_output/RandomSamples/test_input_/gen_start_scale=1/0.png")

FileNotFoundError: [Errno 2] No such file or directory: '/work/vajira/DATA/sinGAN_polyps/test_output/RandomSamples/test_input_/gen_start_scale=1/0.png'

In [ ]:
img